# Lab 4: Urban Heat Island Detection using Thermal Satellite Imagery

## Goal
In this exercise, you will:
1. Select and compare two thermal satellite images from Kraków (or another city that has recently undergone intensive, thoughtless concrete development) – one from **2013** and one from **2024**.
2. Each image must be acquired on a **hot summer day** (T > 27°C based on IMGW meteorological data).
3. Images must have **low cloud cover** (< 20%).
4. Visualize thermal data and analyze surface temperature differences (Urban Heat Island effect).

## Task 1: Area of Interest (AOI)
- Define an AOI over the Reduta Street area in Kraków.
- Use `ee.Geometry.Polygon` or `ee.Geometry.Rectangle`.

Tutaj próbowałem szukać innych ulic które w 2013 jeszcze nie były zabudowane ale w końcu wybrałem Reduta bo na niej widać największą zmianę (2013-łąki, 2024-gęsta zabudowa)

In [ ]:
import ee
import geemap
ee.Initialize(project='ee-jsumara')

# Define an AOI using ee.Geometry.Rectangle
aoi = ee.Geometry.Rectangle([19.98066, 50.09574, 19.98761, 50.09400])  # Coordinates for Reduta Street area in Kraków

# Print the AOI to verify
print(aoi.getInfo())

# Create a map centered on the AOI
Map = geemap.Map(center=(50.095, 19.985), zoom=15)
Map.addLayer(aoi, {'color': 'red'}, "AOI")
Map

{'type': 'Polygon', 'coordinates': [[[19.98066, 50.094], [19.98761, 50.094], [19.98761, 50.09574], [19.98066, 50.09574], [19.98066, 50.094]]]}


Map(center=[50.095, 19.985], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

## Task 2: Download and Analyze IMGW Meteorological Data
- Visit: [IMGW Archive](https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/)
- Navigate to:
  - `dane_meteorologiczne/dobowe/synop/2013/`
  - `dane_meteorologiczne/dobowe/synop/2024/`
- Download `s_d_tmax.csv` for both years.
- Filter the rows for **station ID 12566 (e.g. Kraków-Balice)**.
- Identify days in **July or August** with **TMAX > 27°C**.

Pobieram csv bo api jest jakieś dziwne i średnio opisane... Id stacji kraków-balice: 566. Max temp kolumna 5

In [ ]:
import pandas as pd

def extract_hot_days(filepath, station_name="KRAKÓW-BALICE", tmax_threshold=27, summer_months=(7, 8)):

    df = pd.read_csv(filepath, sep=",", encoding="cp1250", header=None)
    df.columns = [f"col_{i}" for i in range(df.shape[1])]
    df_dates = df.rename(columns={
        "col_2": "year",
        "col_3": "month",
        "col_4": "day"
    })
    df["date"] = pd.to_datetime(df_dates[["year", "month", "day"]], errors="coerce")
    df["tmax"] = pd.to_numeric(df["col_5"], errors="coerce")
    hot_days = df[
        (df["date"].dt.month.isin(summer_months)) &
        (df["tmax"] > tmax_threshold)
    ][["date", "tmax"]]

    return hot_days.reset_index(drop=True)

In [28]:
hot_days_2013 = extract_hot_days("D:/RS/rs-summer-2025-labs-jakubsumara/lab4dane/s_d_566_2013.csv")
print(hot_days_2013)
hot_days_2024 = extract_hot_days("D:/RS/rs-summer-2025-labs-jakubsumara/lab4dane/s_d_566_2024.csv")
print(hot_days_2024)

         date  tmax
0  2013-07-03  28.3
1  2013-07-04  28.3
2  2013-07-05  27.1
3  2013-07-10  27.7
4  2013-07-22  27.7
5  2013-07-26  28.9
6  2013-07-27  30.5
7  2013-07-28  35.0
8  2013-07-29  35.5
9  2013-07-30  29.6
10 2013-08-02  29.9
11 2013-08-03  31.4
12 2013-08-04  29.7
13 2013-08-05  30.8
14 2013-08-06  33.4
15 2013-08-07  36.1
16 2013-08-08  37.3
17 2013-08-09  29.6
18 2013-08-17  27.9
19 2013-08-18  30.6
20 2013-08-19  31.7
         date  tmax
0  2024-07-01  28.9
1  2024-07-06  30.1
2  2024-07-07  27.4
3  2024-07-09  30.6
4  2024-07-10  34.4
5  2024-07-11  29.8
6  2024-07-12  30.0
7  2024-07-13  29.2
8  2024-07-15  30.1
9  2024-07-16  31.2
10 2024-07-17  29.2
11 2024-07-18  28.1
12 2024-07-19  28.0
13 2024-07-21  29.0
14 2024-07-22  31.3
15 2024-07-27  30.6
16 2024-07-28  28.0
17 2024-07-31  29.4
18 2024-08-02  27.1
19 2024-08-08  27.6
20 2024-08-10  27.3
21 2024-08-11  27.3
22 2024-08-12  27.4
23 2024-08-13  27.1
24 2024-08-14  28.6
25 2024-08-15  31.5
26 2024-08-16  31.1


2013 rok: widać super okres od 2 do 9 sierpnia, gdzie średnia temperatura przekracza 30 stopni Celsjusza.
2024 rok: tutaj wybiorę jakieś zobrazowanie w datach od 6 do 22 lipca.

## Task 3: Select Landsat 8 Images Matching These Dates
- In Earth Engine, use `LANDSAT/LC08/C02/T1_L2` collection.
- Apply filters:
  - `.filterBounds(aoi)`
  - `.filterDate()` for the matching day
  - `.filterMetadata('CLOUD_COVER', 'less_than', 20)`
- Try to find **one image from 2013** and **one image from 2024** that match your hot days list.

In [ ]:
#Landsat 9 działa od 2021 roku więc wybieram Landsat 8 do obu dat, bo on wciąż wykonuje zdjęcia. Nie ograniczałem zachmurzenia
#bo moje AOI jest małe, poza tym dla 2024 roku są 3 zobrazowania a każdme z nich ma zachmurzenie powyżej 60%, więc sprawdzałem 
#na oko czy AOI jest zakryte
def list_landsat8_images(aoi, start_date, end_date):
    # Filtruj kolekcję Landsat 8
    collection = (
        ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .sort('system:time_start')
    )

    def format_info(image):
        return ee.Feature(None, {
            'id': image.get('system:index'),
            'date': image.date().format('YYYY-MM-dd'),
            'cloud': image.get('CLOUD_COVER')
        })

    image_info_fc = collection.map(format_info)

    # Wyciągnij dane jako listę
    dates = image_info_fc.aggregate_array('date').getInfo()
    ids = image_info_fc.aggregate_array('id').getInfo()
    clouds = image_info_fc.aggregate_array('cloud').getInfo()

    result = [
        {'date': dates[i], 'id': ids[i], 'cloud_cover': clouds[i]}
        for i in range(len(dates))
    ]
    return result

In [38]:
images = list_landsat8_images(aoi, '2013-07-01', '2013-08-31')

for img in images:
    print(f"{img['date']} | ID: {img['id']} | Cloud cover: {img['cloud_cover']}%")

images = list_landsat8_images(aoi, '2024-07-01', '2024-08-31')

for img in images:
    print(f"{img['date']} | ID: {img['id']} | Cloud cover: {img['cloud_cover']}%")

2013-07-06 | ID: LC08_188025_20130706 | Cloud cover: 93.37%
2013-07-22 | ID: LC08_188025_20130722 | Cloud cover: 8.95%
2013-08-07 | ID: LC08_188025_20130807 | Cloud cover: 0.54%
2013-08-23 | ID: LC08_188025_20130823 | Cloud cover: 50.94%
2024-07-04 | ID: LC08_188025_20240704 | Cloud cover: 70.55%
2024-07-20 | ID: LC08_188025_20240720 | Cloud cover: 83.8%
2024-08-21 | ID: LC08_188025_20240821 | Cloud cover: 62.14%


In [56]:
# Funkcja pomocnicza do pobrania obrazu po ID
def get_landsat8_image_by_id(image_id):
    return ee.Image(f'LANDSAT/LC08/C02/T1_L2/{image_id}').clip(aoi)

# Obrazy z wybranych dat
image_2013 = get_landsat8_image_by_id('LC08_188025_20130807')
image_2024_1 = get_landsat8_image_by_id('LC08_188025_20240704')
image_2024_2 = get_landsat8_image_by_id('LC08_188025_20240720')
#image_2024_3 = get_landsat8_image_by_id('LC08_188025_20240821')


# Parametry RGB (prawdziwy kolor: B4, B3, B2)
vis_params = {
    'min': 0,
    'max': 30000,
    'bands': ['SR_B4', 'SR_B3', 'SR_B2']
}

# Tworzenie mapy porównawczej
Map = geemap.Map(center=[50.093, 20.015], zoom=14)
Map.addLayer(image_2013, vis_params, 'Landsat 8 - 2013-08-07 (0.5% cloud)')
Map.addLayer(image_2024_1, vis_params, 'Landsat 8 - 2024-07-04 (70.55% cloud)')
Map.addLayer(image_2024_2, vis_params, 'Landsat 8 - 2024-07-20 (83.8% cloud)')
#Map.addLayer(image_2024_3, vis_params, 'Landsat 8 - 2024-07-20 (62.14% cloud)')
Map.addLayer(aoi, {}, 'AOI')
Map

Map(center=[50.093, 20.015], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

## Task 4: Process Thermal Band (ST_B10)
- Convert Band 10 to Brightness Temperature (Kelvin):
  `TB = ST_B10 * 0.00341802 + 149.0`
- Create a visualization of each image using the same color scale.

In [64]:
# Function to calculate Brightness Temperature (Kelvin) from ST_B10
def calculate_brightness_temperature(image):
    return image.select('ST_B10').multiply(0.00341802).add(149.0).rename('Brightness_Temperature')

# Calculate Brightness Temperature for 2013 and 2024 images
temp_2013 = calculate_brightness_temperature(image_2013)
temp_2024 = calculate_brightness_temperature(image_2024_2)

# Visualization parameters for Brightness Temperature
vis_temp = {
    'min': 260,
    'max': 325,
    'palette': ['blue', 'cyan', 'green', 'yellow', 'red']
}

# Add layers to the map
Map.addLayer(temp_2013, vis_temp, 'Brightness Temperature 2013')
Map.addLayer(temp_2024, vis_temp, 'Brightness Temperature 2024')
Map
# Function to convert Kelvin to Celsius
def kelvin_to_celsius(image):
    return image.subtract(273.15).rename('Temperature_Celsius')

# Convert Brightness Temperature to Celsius
temp_2013_celsius = kelvin_to_celsius(temp_2013)
temp_2024_celsius = kelvin_to_celsius(temp_2024)

# Visualization parameters for Celsius Temperature
vis_temp_celsius = {
    'min': -2,  # 260K in Celsius
    'max': 41.2,   # 325K in Celsius
    'palette': ['blue', 'cyan', 'green', 'yellow', 'red']
}

# Add Celsius layers to the map
Map.addLayer(temp_2013_celsius, vis_temp_celsius, 'Temperature 2013 (Celsius)')
Map.addLayer(temp_2024_celsius, vis_temp_celsius, 'Temperature 2024 (Celsius)')
Map

Map(bottom=89234.0, center=[49.974188803210936, 20.20523071289063], controls=(WidgetControl(options=['position…

Wyniki są odwrotne niż spodziewane... Temp w 2013 to około 312 stopni a w 2024 275...

In [58]:
def check_temperature_range(image, aoi):
    temp_image = image.select('ST_B10').multiply(0.00341802).add(149.0)
    stats = temp_image.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=aoi,
        scale=30,
        bestEffort=True
    )
    return stats.getInfo()

print("Zakres temperatur 2013:", check_temperature_range(image_2013, aoi))
print("Zakres temperatur 2024:", check_temperature_range(image_2024_2, aoi))

Zakres temperatur 2013: {'ST_B10_max': 314.33988146, 'ST_B10_min': 310.34421608}
Zakres temperatur 2024: {'ST_B10_max': 278.61815444, 'ST_B10_min': 270.44566862}


## Task 5: Compare and Interpret
- Compare the two maps.
- Optionally calculate difference: `TB_2024 - TB_2013`
- Discuss: did the surface temperature increase in the area?
- Is there evidence of an Urban Heat Island effect related to development?

## Analiza wyników

Wyniki są sprzeczne z oczekiwaniami: w 2013 temperatura wynosi ~312 K (~39°C), a w 2024 ~275 K (~2°C). Spodziewano się wzrostu temperatury z powodu efektu miejskiej wyspy ciepła.

### Możliwe przyczyny:
1. **Zachmurzenie**: Obrazy z 2024 mają wysokie zachmurzenie (70.55%, 83.8%), co może zaniżać odczyty (badany obszar nie jest zachmurzony).
2. **Błąd przetwarzania**: Możliwe problemy z danymi ST_B10 lub skalowaniem.
3. **Warunki atmosferyczne**: Różnice w wilgotności i wiatrach mogą wpływać na wyniki.
4. **Wybór obrazów**: Ograniczona liczba zobrazowań z 2024.
5. **Artefakty danych**: Możliwe błędy w pasmach termalnych Landsat 8.

### Wnioski:
- Wysokie zachmurzenie zniekształca wyniki.
- Należy zweryfikować przetwarzanie danych i rozważyć inne obrazy z mniejszym zachmurzeniem.
